In [39]:
# Импортируем нужные нам библиотеки
from sklearn import (metrics, model_selection as skms, naive_bayes,
                     neighbors, tree, gaussian_process, neural_network, 
                     preprocessing, linear_model, svm)
import pandas as pd
import numpy as np

from autogluon.tabular import TabularDataset, TabularPredictor

In [40]:
# Загружаем CSV файл и удаляем неинтересующие нас колонки
dataframe = pd.read_csv('marketing_campaign.csv', delimiter='\t')
dataframe = dataframe.drop(['ID','Dt_Customer'], axis=1)
dataframe.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,1957,Graduation,Single,58138.0,0,0,58,635,88,546,...,7,0,0,0,0,0,0,3,11,1
1,1954,Graduation,Single,46344.0,1,1,38,11,1,6,...,5,0,0,0,0,0,0,3,11,0
2,1965,Graduation,Together,71613.0,0,0,26,426,49,127,...,4,0,0,0,0,0,0,3,11,0
3,1984,Graduation,Together,26646.0,1,0,26,11,4,20,...,6,0,0,0,0,0,0,3,11,0
4,1981,PhD,Married,58293.0,1,0,94,173,43,118,...,5,0,0,0,0,0,0,3,11,0


In [41]:
# Производим кодирование текстовых признаков
scaler_encoder = preprocessing.OrdinalEncoder(encoded_missing_value=-1)
dataframe[['Education', 'Marital_Status']]=scaler_encoder.fit_transform(dataframe[['Education', 'Marital_Status']])

In [42]:
# Удаляем строки в которых остались значения не являющиеся числом
dataframe = dataframe.dropna()
np.argwhere(np.isnan(dataframe))

array([], shape=(0, 2), dtype=int64)

In [43]:
# Разбиваем наши данные на тренировочную и тестовую выборку в соотношении 80/20
(dataframe_train_ftrs, dataframe_test_ftrs) = skms.train_test_split(dataframe,
                                                                   test_size=0.2,
                                                                   random_state=42)

In [44]:
# Определяем колонку с признаком
label = 'Response'

In [45]:
predictor = TabularPredictor(label=label).fit(dataframe_train_ftrs)

No path specified. Models will be saved in: "AutogluonModels\ag-20241208_163435"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.1
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          32
Memory Avail:       42.50 GB / 63.69 GB (66.7%)
Disk Space Avail:   580.82 GB / 951.64 GB (61.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'        

In [46]:
y_pred = predictor.predict(dataframe_test_ftrs.drop(columns=[label]))
y_pred.head()

961     0
229     0
1093    0
427     0
1650    0
Name: Response, dtype: int64

In [47]:
predictor.evaluate(dataframe_test_ftrs, silent=True)

{'accuracy': 0.8738738738738738,
 'balanced_accuracy': 0.6902550244891066,
 'mcc': 0.4247422322749741,
 'roc_auc': 0.8728255362269888,
 'f1': 0.4909090909090909,
 'precision': 0.5625,
 'recall': 0.43548387096774194}

In [48]:
predictor.leaderboard(dataframe_test_ftrs)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,0.891892,0.890141,accuracy,0.015639,0.010346,1.029865,0.015639,0.010346,1.029865,1,True,3
1,CatBoost,0.882883,0.898592,accuracy,0.008473,0.000000,1.942304,0.008473,0.000000,1.942304,1,True,7
2,ExtraTreesEntr,0.880631,0.878873,accuracy,0.091552,0.069667,0.648226,0.091552,0.069667,0.648226,1,True,9
3,NeuralNetFastAI,0.878378,0.876056,accuracy,0.017138,0.000000,1.353909,0.017138,0.000000,1.353909,1,True,10
4,RandomForestEntr,0.878378,0.881690,accuracy,0.085666,0.069915,0.650865,0.085666,0.069915,0.650865,1,True,6
5,XGBoost,0.876126,0.887324,accuracy,0.025135,0.000000,0.715636,0.025135,0.000000,0.715636,1,True,11
6,RandomForestGini,0.876126,0.881690,accuracy,0.106437,0.083191,0.699134,0.106437,0.083191,0.699134,1,True,5
7,WeightedEnsemble_L2,0.873874,0.898592,accuracy,0.000832,0.011706,1.022234,0.000000,0.000000,0.101222,2,True,13
8,LightGBM,0.873874,0.898592,accuracy,0.000832,0.011706,0.921012,0.000832,0.011706,0.921012,1,True,4
9,KNeighborsUnif,0.871622,0.825352,accuracy,0.032901,0.033077,0.008632,0.032901,0.033077,0.008632,1,True,1
